In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from line_profiler import LineProfiler
from numba import jit
from utils.plotting_functions import *
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

# Q-learning

In [17]:
# np.random.seed(2020) # Price cycle where firm 1 free rides
np.random.seed(2021) # Optimal collusive equilibrium
# np.random.seed(2022) # Collusive, but not joint profit-maximizing - collusive on p=0.2
T = 10 # Iterations
delta = 0.95 # Discount factor
alpha = 0.3 # Learning rate
theta = 0.0000275 # Parameter for exploration
k = 6 # Number of equally sized price intervals
s = np.round(np.arange(k + 1) / (np.zeros(k + 1) + k), 3)
t = np.arange(T + 2) + 1
epsilon = np.array([(1 - theta)**i for i in t])
# u = np.random.uniform(size=(T + 2))

# def select_action(Q, current_s, s, epsilon):
#     if np.random.uniform() < epsilon:
#         return np.random.choice(s)
#     else:
#         max_idx = np.argmax(Q[np.where(s == current_s), :])
#         return s[max_idx]

def select_action(Q, current_s, s, epsilon):
    sum_Q = np.sum(np.exp(Q[np.where(s == current_s), :]) / epsilon)
    QQ = np.exp(Q[np.where(s == current_s), :]) / epsilon
    probs = QQ / sum_Q
    return sum_Q

def profit(p_i, p_j):
    if p_i < p_j:
        d = 1 - p_i
    elif p_i == p_j:
        d =  0.5 * (1 - p_i)
    else:
        d = 0

    return p_i * d

def sequential_Q_learning(s, delta, epsilon, alpha):
    # Initialize
    Q_1 = np.zeros((len(s), len(s)))
    Q_2 = np.zeros((len(s), len(s)))

    p_1 = 0
    p_2 = 1 # Set to 1 to mimic that firm 2 has not entered market yet
    p_1_prime = select_action(Q_1, p_2, s, epsilon[0])
    p_2_prime = select_action(Q_2, p_1_prime, s, epsilon[1])


    # Average 1000 period profits
    profits_1 = np.empty(1000)
    profits_1[:] = np.nan
    profits_2 = np.empty(1000)
    profits_2[:] = np.nan
    profits_1_avgs = np.zeros(T)
    profits_2_avgs = np.zeros(T)

    for t in range(2, len(epsilon)):
        if (t % 2) == 0:
            p_1 = p_1_prime
            pi = profit(p_1, p_2)
            pi_prime = profit(p_1, p_2_prime)
            r_t = pi + delta * pi_prime
            a_idx = np.where(s == p_1)
            p_2_prime_idx = np.where(s == p_2_prime)
            p_2_idx = np.where(s == p_2)
            max_Q = max(Q_1[p_2_prime_idx[0][0], :])
            Q_1[p_2_idx, a_idx] = Q_1[p_2_idx, a_idx] + alpha * (r_t + delta**2 * max_Q - Q_1[p_2_idx, a_idx])
            p_1_prime = select_action(Q_1, p_2_prime, s, epsilon[t])
            profits_1 = np.concatenate(([pi], profits_1[0:-1]))
            profits_1_avgs[t - 2] = np.nanmean(profits_1)
            pi_2 = profit(p_2, p_1)
            profits_2 = np.concatenate(([pi_2], profits_2[0:-1]))
            profits_2_avgs[t - 2] = np.nanmean(profits_2)
        else:
            p_2 = p_2_prime
            pi = profit(p_2, p_1)
            pi_prime = profit(p_2, p_1_prime)
            r_t = pi + delta * pi_prime
            a_idx = np.where(s == p_2)
            p_1_prime_idx = np.where(s == p_1_prime)
            p_1_idx = np.where(s == p_1)
            max_Q = max(Q_2[p_1_prime_idx[0][0], :])
            Q_2[p_1_idx, a_idx] = Q_2[p_1_idx, a_idx] + alpha * (r_t + delta**2 * max_Q - Q_2[p_1_idx, a_idx])
            p_2_prime = select_action(Q_2, p_1_prime, s, epsilon[t])
            profits_2 = np.concatenate(([pi], profits_2[0:-1]))
            profits_2_avgs[t - 2] = np.nanmean(profits_2)
            pi_1 = profit(p_1, p_2)
            profits_1 = np.concatenate(([pi_1], profits_1[0:-1]))
            profits_1_avgs[t - 2] = np.nanmean(profits_1)

    return Q_1, Q_2, profits_1_avgs, profits_2_avgs

def sequential_SARSA_learning(s, delta, epsilon, alpha):
    # Initialize
    Q_1 = np.zeros((len(s), len(s)))
    Q_2 = np.zeros((len(s), len(s)))

    p_1 = 0
    p_2 = 1 # Set to 1 to mimic that firm 2 has not entered market yet
    p_1_prime = select_action(Q_1, p_2, s, epsilon[0])
    p_2_prime = select_action(Q_2, p_1_prime, s, epsilon[1])

    # Average 1000 period profits
    profits_1 = np.empty(1000)
    profits_1[:] = np.nan
    profits_2 = np.empty(1000)
    profits_2[:] = np.nan
    profits_1_avgs = np.zeros(T)
    profits_2_avgs = np.zeros(T)

    for t in range(2, len(epsilon)):
        if (t % 2) == 0:
            p_1 = p_1_prime
            pi = profit(p_1, p_2)
            pi_prime = profit(p_1, p_2_prime)
            r_t = pi + delta * pi_prime
            a_idx = np.where(s == p_1)
            p_2_prime_idx = np.where(s == p_2_prime)
            p_2_idx = np.where(s == p_2)
            p_1_prime = select_action(Q_1, p_2_prime, s, epsilon[t])
            p_1_prime_idx = np.where(s == p_1_prime)[0][0]
            Q_1[p_2_idx, a_idx] = Q_1[p_2_idx, a_idx] + alpha * (r_t + delta**2 * Q_1[p_2_prime_idx[0][0], p_1_prime_idx] - Q_1[p_2_idx, a_idx])
            profits_1 = np.concatenate(([pi], profits_1[0:-1]))
            profits_1_avgs[t - 2] = np.nanmean(profits_1)
            pi_2 = profit(p_2, p_1)
            profits_2 = np.concatenate(([pi_2], profits_2[0:-1]))
            profits_2_avgs[t - 2] = np.nanmean(profits_2)
        else:
            p_2 = p_2_prime
            pi = profit(p_2, p_1)
            pi_prime = profit(p_2, p_1_prime)
            r_t = pi + delta * pi_prime
            a_idx = np.where(s == p_2)
            p_1_prime_idx = np.where(s == p_1_prime)
            p_1_idx = np.where(s == p_1)
            p_2_prime = select_action(Q_2, p_1_prime, s, epsilon[t])
            p_2_prime_idx = np.where(s == p_2_prime)[0][0]
            Q_2[p_1_idx, a_idx] = Q_2[p_1_idx, a_idx] + alpha * (r_t + delta**2 * Q_2[p_1_prime_idx[0][0], p_2_prime_idx] - Q_2[p_1_idx, a_idx])
            profits_2 = np.concatenate(([pi], profits_2[0:-1]))
            profits_2_avgs[t - 2] = np.nanmean(profits_2)
            pi_1 = profit(p_1, p_2)
            profits_1 = np.concatenate(([pi_1], profits_1[0:-1]))
            profits_1_avgs[t - 2] = np.nanmean(profits_1)

    return Q_1, Q_2, profits_1_avgs, profits_2_avgs

Q_1, Q_2, profits_1_avgs, profits_2_avgs = sequential_Q_learning(s, delta, epsilon, alpha)

# lp = LineProfiler()
# lp_wrapper = lp(sequential_Q_learning)
# lp_wrapper(s, delta, epsilon, alpha)
# lp.print_stats()
    

# Boltzman exploration policy

In [ ]:
tau = np.logspace(1, -6, 500002) # temperature parameter in Boltzman policy

def select_action(Q, current_s, s, tau):

    sum_Q = np.sum((np.exp(Q[np.where(s == current_s), :]) / tau))

In [3]:
T = 500000 # Iterations
delta = 0.95 # Discount factor
alpha = 0.3 # Learning rate
theta = 0.0000275 # Parameter for exploration
k = 6 # Number of equally sized price intervals
s = np.round(np.arange(k + 1) / (np.zeros(k + 1) + k), 3)
t = np.arange(T + 2) + 1
epsilon = np.array([(1 - theta)**i for i in t])

@jit(nopython=True)
def select_action(Q, current_s, s, epsilon, u):
    if u < epsilon:
        return np.random.choice(s)
    else:
        max_idx = np.argmax(Q[np.where(s == current_s)[0][0], :])
        return s[max_idx]

@jit(nopython=True)
def profit(p_i, p_j):
    if p_i < p_j:
        d = 1 - p_i
    elif p_i == p_j:
        d =  0.5 * (1 - p_i)
    else:
        d = 0

    return p_i * d

def mode(x):
    """Returns mode of an array"""
    values, counts = np.unique(x, return_counts=True)
    m = counts.argmax()
    return values[m]

@jit(nopython=True)
def sequential_Q_learning(s, delta, epsilon, alpha, u):
    # Initialize
    Q_1 = np.zeros((len(s), len(s)))
    Q_2 = np.zeros((len(s), len(s)))

    p_1 = 0
    p_2 = 1 # Set to 1 to mimic that firm 2 has not entered market yet
    p_1_prime = select_action(Q_1, p_2, s, epsilon[0], u[0])
    p_2_prime = select_action(Q_2, p_1_prime, s, epsilon[1], u[1])    

    # Average 1000 period profits
    profits_1 = np.empty(1000)
    profits_1[:] = np.nan
    profits_2 = np.empty(1000)
    profits_2[:] = np.nan
    profits_1_avgs = np.zeros(T)
    profits_2_avgs = np.zeros(T)

    for t in range(2, len(epsilon)):
        if (t % 2) == 0:
            p_1 = p_1_prime
            pi = profit(p_1, p_2)
            pi_prime = profit(p_1, p_2_prime)
            r_t = pi + delta * pi_prime
            a_idx = np.where(s == p_1)[0][0]
            p_2_prime_idx = np.where(s == p_2_prime)[0][0]
            p_2_idx = np.where(s == p_2)[0][0]
            max_Q = max(Q_1[p_2_prime_idx, :])
            Q_1[p_2_idx, a_idx] = Q_1[p_2_idx, a_idx] + alpha * (r_t + delta**2 * max_Q - Q_1[p_2_idx, a_idx])
            p_1_prime = select_action(Q_1, p_2_prime, s, epsilon[t], u[t])
            profits_1 = np.concatenate((np.array([pi]), profits_1[0:-1]))
            profits_1_avgs[t - 2] = np.nanmean(profits_1)
            pi_2 = profit(p_2, p_1)
            profits_2 = np.concatenate((np.array([pi_2]), profits_2[0:-1]))
            profits_2_avgs[t - 2] = np.nanmean(profits_2)
        else:
            p_2 = p_2_prime
            pi = profit(p_2, p_1)
            pi_prime = profit(p_2, p_1_prime)
            r_t = pi + delta * pi_prime
            a_idx = np.where(s == p_2)[0][0]
            p_1_prime_idx = np.where(s == p_1_prime)[0][0]
            p_1_idx = np.where(s == p_1)[0][0]
            max_Q = max(Q_2[p_1_prime_idx, :])
            Q_2[p_1_idx, a_idx] = Q_2[p_1_idx, a_idx] + alpha * (r_t + delta**2 * max_Q - Q_2[p_1_idx, a_idx])
            p_2_prime = select_action(Q_2, p_1_prime, s, epsilon[t], u[t])
            profits_2 = np.concatenate((np.array([pi]), profits_2[0:-1]))
            profits_2_avgs[t - 2] = np.nanmean(profits_2)
            pi_1 = profit(p_1, p_2)
            profits_1 = np.concatenate((np.array([pi_1]), profits_1[0:-1]))
            profits_1_avgs[t - 2] = np.nanmean(profits_1)

    return Q_1, Q_2, profits_1_avgs, profits_2_avgs, profits_1, profits_2


@jit(nopython=True)
def sequential_SARSA_learning(s, delta, epsilon, alpha, u):
    # Initialize
    Q_1 = np.zeros((len(s), len(s)))
    Q_2 = np.zeros((len(s), len(s)))

    p_1 = 0
    p_2 = 1 # Set to 1 to mimic that firm 2 has not entered market yet
    p_1_prime = select_action(Q_1, p_2, s, epsilon[0], u[0])
    p_2_prime = select_action(Q_2, p_1_prime, s, epsilon[1], u[0])    

    # Average 1000 period profits
    profits_1 = np.empty(1000)
    profits_1[:] = np.nan
    profits_2 = np.empty(1000)
    profits_2[:] = np.nan
    profits_1_avgs = np.zeros(T)
    profits_2_avgs = np.zeros(T)

    for t in range(2, len(epsilon)):
        if (t % 2) == 0:
            p_1 = p_1_prime
            pi = profit(p_1, p_2)
            pi_prime = profit(p_1, p_2_prime)
            r_t = pi + delta * pi_prime
            a_idx = np.where(s == p_1)[0][0]
            p_2_prime_idx = np.where(s == p_2_prime)[0][0]
            p_2_idx = np.where(s == p_2)[0][0]
            p_1_prime = select_action(Q_1, p_2_prime, s, epsilon[t], u[t])
            p_1_prime_idx = np.where(s == p_1_prime)[0][0]
            Q_1[p_2_idx, a_idx] = Q_1[p_2_idx, a_idx] + alpha * (r_t + delta**2 * Q_1[p_2_prime_idx, p_1_prime_idx] - Q_1[p_2_idx, a_idx])
            profits_1 = np.concatenate((np.array([pi]), profits_1[0:-1]))
            profits_1_avgs[t - 2] = np.nanmean(profits_1)
            pi_2 = profit(p_2, p_1)
            profits_2 = np.concatenate((np.array([pi_2]), profits_2[0:-1]))
            profits_2_avgs[t - 2] = np.nanmean(profits_2)
        else:
            p_2 = p_2_prime
            pi = profit(p_2, p_1)
            pi_prime = profit(p_2, p_1_prime)
            r_t = pi + delta * pi_prime
            a_idx = np.where(s == p_2)[0][0]
            p_1_prime_idx = np.where(s == p_1_prime)[0][0]
            p_1_idx = np.where(s == p_1)[0][0]
            p_2_prime = select_action(Q_2, p_1_prime, s, epsilon[t], u[t])
            p_2_prime_idx = np.where(s == p_2_prime)[0][0]
            Q_2[p_1_idx, a_idx] = Q_2[p_1_idx, a_idx] + alpha * (r_t + delta**2 * Q_2[p_1_prime_idx, p_2_prime_idx] - Q_2[p_1_idx, a_idx])
            profits_2 = np.concatenate((np.array([pi]), profits_2[0:-1]))
            profits_2_avgs[t - 2] = np.nanmean(profits_2)
            pi_1 = profit(p_1, p_2)
            profits_1 = np.concatenate((np.array([pi_1]), profits_1[0:-1]))
            profits_1_avgs[t - 2] = np.nanmean(profits_1)

    return Q_1, Q_2, profits_1_avgs, profits_2_avgs



# Simulation with n repetitions
n = 100 # Repetitions
u = np.random.uniform(size=(n, T + 2))
profits_1_avgs = np.zeros((n, T))
profits_2_avgs = np.zeros((n, T))
opt_profit = 0.125 # Joint maximizing profit

NE_counter = 0 # Nash Equilibrium Counter
opt_NE_counter = 0 # Optimal Nash Equilibium Counter

for i in range(n):
    _, _, profits_1_avgs[i, :], profits_2_avgs[i, :], profits_1, profits_2 = sequential_Q_learning(s, delta, epsilon, alpha, u[i, :])
    mode_1, mode_2 = mode(profits_1), mode(profits_2)
    if sum(profits_1 == mode_1) > 995 and sum(profits_2 == mode_2) > 995:
        NE_counter += 1
        if mode_1 == mode_2 == opt_profit:
            opt_NE_counter += 1
    print(i)

# Avg. profits over 1000 time periods
profits_1_avgs = np.mean(profits_1_avgs, axis=0)
profits_2_avgs = np.mean(profits_2_avgs, axis=0)

# Shares of Nash Equilibria
NE_shares = NE_counter / n
opt_NE_shares = opt_NE_counter / n

print("Shares of Nash Equilibria", NE_shares)
print("Shares of optimal Nash Equilibria", opt_NE_shares)
print("NE_counter", NE_counter)
print("NE_counter", opt_NE_counter)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
Shares of Nash Equilibria 0.63
Shares of optimal Nash Equilibria 0.24
NE_counter 63
NE_counter 24


In [92]:
%matplotlib tk
time = np.arange(T)
title = 'Average Profitability for 1000 runs, SARSA'
fig = avg_profits_plot(time[1:], profits_1_avgs[1:], profits_2_avgs[1:], title=title)

# Synchronous pricing - Asker Fershtman

In [15]:
T = 500000 # Iterations
delta = 0.95 # Discount factor
alpha = 0.3 # Learning rate
theta = 0.0000275 # Parameter for exploration
k = 6 # Number of equally sized price intervals
s = np.round(np.arange(k + 1) / (np.zeros(k + 1) + k), 3)
t = np.arange(T + 2) + 1
epsilon = np.array([(1 - theta)**i for i in t])

@jit(nopython=True)
def select_action(Q, current_s, s, epsilon, u):
    if u < epsilon:
        return np.random.choice(s)
    else:
        max_idx = np.argmax(Q[np.where(s == current_s)[0][0], :])
        return s[max_idx]

@jit(nopython=True)
def profit(p_i, p_j):
    if p_i < p_j:
        d = 1 - p_i
    elif p_i == p_j:
        d =  0.5 * (1 - p_i)
    else:
        d = 0

    return p_i * d

def mode(x):
    """Returns mode of an array"""
    values, counts = np.unique(x, return_counts=True)
    m = counts.argmax()
    return values[m]

@jit(nopython=True)
def sequential_Q_learning(s, delta, epsilon, alpha, u):
    # Initialize
    Q_1 = np.zeros((len(s), len(s)))
    Q_2 = np.zeros((len(s), len(s)))

    p_1 = 0
    p_2 = 1 # Set to 1 to mimic that firm 2 has not entered market yet
    p_1_prime = select_action(Q_1, p_2, s, epsilon[0], u[0])
    p_2_prime = select_action(Q_2, p_1_prime, s, epsilon[1], u[1])    
    r_t = np.zeros(len(s))

    # Average 1000 period profits
    profits_1 = np.empty(1000)
    profits_1[:] = np.nan
    profits_2 = np.empty(1000)
    profits_2[:] = np.nan
    profits_1_avgs = np.zeros(T)
    profits_2_avgs = np.zeros(T)

    for t in range(2, len(epsilon)):
        if (t % 2) == 0:
            p_1 = p_1_prime
            for i in range(len(s)):
                pi = profit(p_1, p_2)
                pi_prime = profit(p_1, p_2_prime)
                r_t[i] = pi + delta * pi_prime
            a_idx = np.where(s == p_1)[0][0]
            p_2_prime_idx = np.where(s == p_2_prime)[0][0]
            p_2_idx = np.where(s == p_2)[0][0]
            max_Q = max(Q_1[p_2_prime_idx, :])
            Q_1[p_2_idx, :] = Q_1[p_2_idx, :] + alpha * (r_t + delta**2 * max_Q - Q_1[p_2_idx, :])
            p_1_prime = select_action(Q_1, p_2_prime, s, epsilon[t], u[t])
            profits_1 = np.concatenate((np.array([pi]), profits_1[0:-1]))
            profits_1_avgs[t - 2] = np.nanmean(profits_1)
            pi_2 = profit(p_2, p_1)
            profits_2 = np.concatenate((np.array([pi_2]), profits_2[0:-1]))
            profits_2_avgs[t - 2] = np.nanmean(profits_2)
        else:
            p_2 = p_2_prime
            for i in range(len(s)):
                pi = profit(p_2, p_1)
                pi_prime = profit(p_2, p_1_prime)
                r_t[i] = pi + delta * pi_prime
            a_idx = np.where(s == p_2)[0][0]
            p_1_prime_idx = np.where(s == p_1_prime)[0][0]
            p_1_idx = np.where(s == p_1)[0][0]
            max_Q = max(Q_2[p_1_prime_idx, :])
            Q_2[p_1_idx, :] = Q_2[p_1_idx, :] + alpha * (r_t + delta**2 * max_Q - Q_2[p_1_idx, :])
            p_2_prime = select_action(Q_2, p_1_prime, s, epsilon[t], u[t])
            profits_2 = np.concatenate((np.array([pi]), profits_2[0:-1]))
            profits_2_avgs[t - 2] = np.nanmean(profits_2)
            pi_1 = profit(p_1, p_2)
            profits_1 = np.concatenate((np.array([pi_1]), profits_1[0:-1]))
            profits_1_avgs[t - 2] = np.nanmean(profits_1)

    return Q_1, Q_2, profits_1_avgs, profits_2_avgs, profits_1, profits_2

@jit(nopython=True)
def sequential_SARSA_learning(s, delta, epsilon, alpha, u):
    # Initialize
    Q_1 = np.zeros((len(s), len(s)))
    Q_2 = np.zeros((len(s), len(s)))

    p_1 = 0
    p_2 = 1 # Set to 1 to mimic that firm 2 has not entered market yet
    p_1_prime = select_action(Q_1, p_2, s, epsilon[0], u[0])
    p_2_prime = select_action(Q_2, p_1_prime, s, epsilon[1], u[0])    
    r_t = np.zeros(len(s))

    # Average 1000 period profits
    profits_1 = np.empty(1000)
    profits_1[:] = np.nan
    profits_2 = np.empty(1000)
    profits_2[:] = np.nan
    profits_1_avgs = np.zeros(T)
    profits_2_avgs = np.zeros(T)

    for t in range(2, len(epsilon)):
        if (t % 2) == 0:
            p_1 = p_1_prime
            for i in range(len(s)):
                pi = profit(p_1, p_2)
                pi_prime = profit(p_1, p_2_prime)
                r_t[i] = pi + delta * pi_prime
            a_idx = np.where(s == p_1)[0][0]
            p_2_prime_idx = np.where(s == p_2_prime)[0][0]
            p_2_idx = np.where(s == p_2)[0][0]
            p_1_prime = select_action(Q_1, p_2_prime, s, epsilon[t], u[t])
            p_1_prime_idx = np.where(s == p_1_prime)[0][0]
            Q_1[p_2_idx, :] = Q_1[p_2_idx, :] + alpha * (r_t + delta**2 * Q_1[p_2_prime_idx, p_1_prime_idx] - Q_1[p_2_idx, :])
            profits_1 = np.concatenate((np.array([pi]), profits_1[0:-1]))
            profits_1_avgs[t - 2] = np.nanmean(profits_1)
            pi_2 = profit(p_2, p_1)
            profits_2 = np.concatenate((np.array([pi_2]), profits_2[0:-1]))
            profits_2_avgs[t - 2] = np.nanmean(profits_2)
        else:
            p_2 = p_2_prime
            for i in range(len(s)):
                pi = profit(p_2, p_1)
                pi_prime = profit(p_2, p_1_prime)
                r_t[i] = pi + delta * pi_prime
            a_idx = np.where(s == p_2)[0][0]
            p_1_prime_idx = np.where(s == p_1_prime)[0][0]
            p_1_idx = np.where(s == p_1)[0][0]
            p_2_prime = select_action(Q_2, p_1_prime, s, epsilon[t], u[t])
            p_2_prime_idx = np.where(s == p_2_prime)[0][0]
            Q_2[p_1_idx, :] = Q_2[p_1_idx, :] + alpha * (r_t + delta**2 * Q_2[p_1_prime_idx, p_2_prime_idx] - Q_2[p_1_idx, :])
            profits_2 = np.concatenate((np.array([pi]), profits_2[0:-1]))
            profits_2_avgs[t - 2] = np.nanmean(profits_2)
            pi_1 = profit(p_1, p_2)
            profits_1 = np.concatenate((np.array([pi_1]), profits_1[0:-1]))
            profits_1_avgs[t - 2] = np.nanmean(profits_1)

    return Q_1, Q_2, profits_1_avgs, profits_2_avgs, profits_1, profits_2




# Simulation with n repetitions
n = 2 # Repetitions
u = np.random.uniform(size=(n, T + 2))
profits_1_avgs = np.zeros((n, T))
profits_2_avgs = np.zeros((n, T))
opt_profit = 0.125 # Joint maximizing profit

NE_counter = 0 # Nash Equilibrium Counter
opt_NE_counter = 0 # Optimal Nash Equilibium Counter

for i in range(n):
    _, _, profits_1_avgs[i, :], profits_2_avgs[i, :], profits_1, profits_2 = sequential_SARSA_learning(s, delta, epsilon, alpha, u[i, :])
    mode_1, mode_2 = mode(profits_1), mode(profits_2)
    if sum(profits_1 == mode_1) > 995 and sum(profits_2 == mode_2) > 995:
        NE_counter += 1
        if mode_1 == mode_2 == opt_profit:
            opt_NE_counter += 1
    print(i)

# Avg. profits over 1000 time periods
profits_1_avgs = np.mean(profits_1_avgs, axis=0)
profits_2_avgs = np.mean(profits_2_avgs, axis=0)

# Shares of Nash Equilibria
NE_shares = NE_counter / n
opt_NE_shares = opt_NE_counter / n

print("Shares of Nash Equilibria", NE_shares)
print("Shares of optimal Nash Equilibria", opt_NE_shares)
print("NE_counter", NE_counter)
print("NE_counter", opt_NE_counter)


0
1
Shares of Nash Equilibria 1.0
Shares of optimal Nash Equilibria 0.0
NE_counter 2
NE_counter 0


In [16]:
%matplotlib tk
time = np.arange(T)
title = 'Average Profitability for 1000 runs, Q-Learning'
fig = avg_profits_plot(time[1:], profits_1_avgs[1:], profits_2_avgs[1:], title=title)